In [1]:
import numpy as np
import math
import csv
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
from scipy.spatial import distance

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Read Lidar
def CSV_Read_Lidar_data(data_path):
    rows = []
    with open(data_path, 'r') as file:
        csvreader = csv.reader(file)
        header = next(csvreader)
        for row in csvreader:
            rows.append(row)

    # Create a Series for the header
    header_names = ['angle_min', 'angle_max', 'angle_increment', 'time_increment', 'scan_time', 'range_min', 'range_max']
    Header_info = pd.Series(header, header_names)
    Header_info = Header_info.astype(float)

    # Translation Dataframe
    translation_names = ['trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z', 'rot_w']
    Translation_info = pd.DataFrame(np.array(rows[1::3]).T, translation_names)
    Translation_info = Translation_info.astype(float)

    # Lidar Dataframe
    # X values is the scan number, Y values are the scan pos
    Lidar_info = pd.DataFrame(np.array(rows[2::3]).T)
    Lidar_info.insert(0, "radians", np.array(list(np.multiply(float(Header_info['angle_increment']), np.arange(1, 1081))))) #insert the radians per scan
    Lidar_info = Lidar_info.astype(float)

    return Header_info, Translation_info, Lidar_info

In [3]:
# path to csv data
data_path = 'Hallway_Lidar_data_dinosars2.csv'
Header_info, Translation_info, Lidar_info = CSV_Read_Lidar_data(data_path)

# Parameters for the robot 
# TODO this will be pulled from ros later
velocity_trix = np.array([[0.0], # x y z 1
                         [0.75],
                         [0.0],
                         [1.0]])

b = .235 # distance between robots wheels (meters)
wheel_radius = 0.072 # radius of the actual wheel (meters)

Robot Position Prediction

In [4]:
"""
These are the uncertainties in the are error constants representing the 
nondeterministic parameters of the motor drive and the wheel-floor interaction. Probably can be found in DOC or just tune for it
"""
k_r =  .001
k_l =  .001

# State model Noise covariance matrix w/ assumtion that the variables are independent of each other
Q_t  = np.array([[1.0,  0,   0],
                [  0, 1.0,   0],
                [  0,   0, 1.0]])
F_k_minus_1 = np.array([[1.0,  0,   0],
                        [  0,1.0,   0],
                        [  0,  0, 1.0]])
                        

In [5]:
def position_prediction(pos_t_minus1,delta_sl,delta_sr):

    delta_sl
    delta_sr
    theta_t_minus1 = pos_t_minus1[2,1]
    

    F_k_minus_1

    x_hat = np.empty((3,1))

    # This is previous postion + estimate of future position
    x_hat = pos_t_minus1 + np.array([(delta_sr+delta_sl)/2*math.cos(theta_t_minus1+(delta_sr-delta_sl)/2/b)],
                                    [(delta_sr+delta_sl)/2*math.sin(theta_t_minus1+(delta_sr-delta_sl)/2/b)],
                                    [(delta_sr-delta_sl)/b])
    
    # covarieance of the previouse robot state
    P_t_minus1 = np.array([k_r*abs(delta_sr), 0]
                           [0,k_l*abs(delta_sl) ])
    
    P_hat_t = F_k_minus_1 @ P_t_minus1 @ np.transpose(F_k_minus_1) + F_k_minus_1 @ Q_t @ np.transpose(F_k_minus_1)
    return x_hat, P_hat_t

<>:18: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
<>:18: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
/tmp/ipykernel_309371/2380657657.py:18: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  P_t_minus1 = np.array([k_r*abs(delta_sr), 0]


In [6]:


def postion(previous_pos,inputs_xytheta,):

    new_pos = np.empty((3,1))
    new_pos[0] = inputs_xytheta[0]*.01+velocity_trix[0] + previous_pos[0] #.01 converts to meters
    new_pos[1] = inputs_xytheta[1]*.01+velocity_trix[1] + previous_pos[1]
    new_pos[2] = inputs_xytheta[2]+velocity_trix[2] + previous_pos[2]
    return new_pos    

In [7]:
trans_x = Translation_info.loc['trans_x']
trans_y = Translation_info.loc['trans_y']
trans_z = Translation_info.loc['trans_z']

scan = 1

M = np.array([[1, 0, 0, trans_x[scan]*.01],
             [0, 1, 0, trans_y[scan]*.01],
             [0, 0, 1, trans_z[scan]*.01],
             [0, 0, 0, 1]])

translation_matrix = np.dot(M, velocity_trix)
print(translation_matrix)

[[ 3.07659490e-01]
 [ 4.73303811e-01]
 [-3.49833425e-04]
 [ 1.00000000e+00]]
